# Strong reference 

```
p1 = Object()
p1 = p2

```



# Weak reference 

Does not affect the reference count as far as the memory manager is concern 

```
p1 = Person() # strong reference 
p2 = weakref.ref(p1) |# module weakref
p2() # returns the original object or None if the object has been garbage colected
```

## Strong references counter

When the count goest to zero, the object is destroyed

In [1]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

### Strong references

In [2]:
class Person:
    def __init__(self, name):
        self.name = name 

    def __repr__(self):
        return f'Person(name={self.name})'

In [7]:
p1 = Person('Guido')
p2 = p1 

In [9]:
p1 is p2, id(p1), id(p2) #point strongly to the same object. 

(True, 2169264854016, 2169264854016)

In [16]:
ref_count(address=id(p1))

2

In [17]:
del p2

In [18]:
ref_count(id(p1))

1

In [19]:
p1_id = id(p1)

In [20]:
ref_count(p1_id)

1

In [21]:
del p1

In [22]:
ref_count(p1_id)

0

There is any object at that memory address.

### Weak References

In [24]:
import weakref 

p1 = Person('Lorena')
p1_id = id(p1)

In [25]:
ref_count(p1_id)

1

In [26]:
# Creates strong reference 
p2 = p1 
ref_count(p1_id)

2

Creating the weak reference

In [27]:
weak1 = weakref.ref(p1)

In [28]:
ref_count(p1_id) # has not changed

2

In [29]:
weak1

<weakref at 0x000001F9129FBB00; to 'Person' at 0x000001F912A4C550>

In [31]:
hex(id(p1)).upper()

'0X1F912A4C550'

^ Notice that the memory address from the weak reference is the same than the original object.

The weakref holds a weak reference to another object

In [32]:
weak1 is p1

False

In [35]:
weak1() is p1

True

In [36]:
ref_count(p1_id)

10

Jupyter messed up the references bc I used only `weak1()` in a cell. The correct way to debug should have been `print(weak1())`

In [5]:
import weakref 

p1 = Person('John')
p2 = p1

p1_id = id(p1)

weak1 = weakref.ref(p1)

ref_count(p1_id)

2

In [6]:
del p2
ref_count(p1_id)

1

In [7]:
p3 = weak1() # creates now a strong reference 
ref_count(p1_id)

2

In [8]:
print(weak1())

Person(name=John)


In [9]:
ref_count(p1_id)

2

Deleting all the strong references

In [10]:
del p3
del p1 

ref_count(p1_id)

0

In [11]:
weak1

<weakref at 0x000001F90F8A8770; dead>

Weak references is useful to debug if the object is dead or not. The weak reference stores that information 

In [12]:
result = weak1()

In [14]:
print(result)

None


Most of built in classes in pyhton __does not support weak references__.

Using list as example

In [15]:
l = [1, 2, 3]

try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'list' object


Dictionaries

In [16]:
l = {'a': 1, 'b': 2}

try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'dict' object


Integer, strings.... most of the built in classes does not support weak references, but custom classes does. 

Weak references also provide a dictionary for the objects as a key 

In [18]:
p1 = Person('Gabriel')

d = weakref.WeakKeyDictionary()

ref_count(id(p1))

1

In [19]:
n = {p1: 'Gabriel'}

ref_count(id(p1))

2

In [20]:
del n 
ref_count(id(p1))

1

WeakKeyDictionary stores a weak reference into a dictionary and does not increase the reference count

In [21]:
d[p1] = 'Gabriel'

ref_count(id(p1))

1

In [22]:
weakref.getweakrefcount(p1)

1

In [24]:
d2 = weakref.WeakKeyDictionary()
d2[p1] = 'Gabriel'

ref_count(id(p1)), weakref.getweakrefcount(p1)

(2, 2)

The weak references are storaged in the object itself

In [25]:
p1.__weakref__

<weakref at 0x000001F90F29E7A0; to 'Person' at 0x000001F90F26F6F0>

That is why using slots can generate a problem here. You may not be able to create a proper weak reference to that object.

In [26]:
hex(id(p1))

'0x1f90f26f6f0'

In [27]:
list(d.keyrefs())

[<weakref at 0x000001F90F948C70; to 'Person' at 0x000001F90F26F6F0>]

In [33]:
p = Person('Honer')
ref_count(id(p))

1

In [34]:
d = weakref.WeakKeyDictionary()

d['python'] = 'test'

TypeError: cannot create weak reference to 'str' object

The keys in the weak key dictionary must be a hashable instance of a class 

In [35]:
class Person:
    def __init__(self, name):
        self.name = name 

    def __eq__(self, other):
        return isinstance(other, Person) and other.name 

Now the class Person is NOT hashable because a equality has been stabilshed and not the hash. 

In [36]:
p = Person('Lorena')
d[p] = p.name 

TypeError: unhashable type: 'Person'

To solve, we have to implement the hash mathod...

In [37]:
class Person:
    def __init__(self, name):
        self.name = name 

    def __eq__(self, other):
        return isinstance(other, Person) and other.name 
    
    def __hash__(self):
        return hash(self.name)

In [38]:
p = Person('Lorena')
d[p] = p.name 

In [39]:
list(d.keyrefs())

[<weakref at 0x000001F90F9F7FB0; to 'Person' at 0x000001F90F3EAE40>]